In [1]:
import os
from os.path import join
from os import listdir
import pandas as pd
import numpy as np
import json
import cv2

In [2]:
def load_config(config_path = 'config.json'):
    file = open(config_path,'r')
    return json.load(file)

In [3]:
root = join(join(os.path.abspath(os.curdir)),'data')

In [4]:
train_pathology_path = join(root,'train_pathology.csv')
test_pathology_path = join(root,'test_pathology.csv')
train_plants_path = join(root,'train_plants.csv')
test_plants_path = join(root,'test_plants.csv')

In [5]:
config = load_config()
img_height = config['img_height']
img_width = config['img_width']

In [6]:
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      
    if (img.shape[0]>img.shape[1]):
        img = np.rot90(img)
    
    img = cv2.resize(img,(img_width,img_height),interpolation = cv2.INTER_AREA)
    
    return img

In [7]:
if not os.path.exists(join(root,'images')):
    os.mkdir(join(root,'images'))

In [8]:
from tqdm import tqdm

img_id = 0

def copy_resize_images(df):
    new_paths = []
    for i in tqdm(range(df.shape[0])):
        
        global img_id
        
        new_path = join(root,'images',str(img_id)+'.npy')
        img = load_img(df['path'][i])
        new_paths.append(new_path)
        np.save(new_path,img)
        img_id+=1
        
    df.loc[:,'path'] = new_paths


In [9]:
train_pathology = pd.read_csv(train_pathology_path)
test_pathology = pd.read_csv(test_pathology_path)
train_plants = pd.read_csv(train_plants_path)
test_plants = pd.read_csv(test_plants_path)

In [10]:
copy_resize_images(train_pathology)
copy_resize_images(test_pathology)

train_plants.loc[:,'path'] = train_pathology['path']
test_plants.loc[:,'path'] = test_pathology['path']

100%|██████████| 346/346 [00:09<00:00, 34.67it/s]


In [11]:
train_pathology.to_csv(train_pathology_path,index=False)
test_pathology.to_csv(test_pathology_path,index=False)
train_plants.to_csv(train_plants_path,index=False)
test_plants.to_csv(test_plants_path,index=False)